<a href="https://colab.research.google.com/github/clarissa-souza/Desafio-rox/blob/main/Banco_ROX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Nesta etapa vou fazer a criação, inserção dos dados, alterações necesárias na tabela e as consultas exigidas pelo desafio.

# Bibliotecas

In [2]:
pip install mysql-connector-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 68.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.8/409.8 kB 32.9 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3


In [3]:
pip install pymysql

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.1 MB/s eta 0:00:00


In [22]:
import mysql.connector
from mysql.connector import Error
from IPython.lib import passwd
import requests

In [2]:
import pandas as pd

# Importando para o google cloud
from google.cloud import storage
import os

# Importando biblioteca para acessar a Service account que está no drive
from google.colab import drive

In [26]:
# conector para popular o banco
import pymysql
from sqlalchemy import create_engine

# Acesso ao bucket dos dados tratados

In [4]:
# Montando o acesso ao drive para acessar a Service account
drive.mount ('/content/drive')

# Configurando o caminho da Service account
serviceAccount='/content/drive/MyDrive/Rox/leega-estudos-6d979ec7a228.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS']=serviceAccount

# Instanciar o client
client=storage.Client()

# Receber o nome do bucket na variavel bucket
bucket=client.get_bucket('bucket-rox')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Funções

In [5]:
# função para conectar ao servidor mysql
def create_conexao(servidor,usuario,senha):
    connection = None
    try:
        connection = mysql.connector.connect(
          host = servidor,
          user = usuario,
          passwd = senha,
        )
        print("MySQL Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")

    return connection

In [6]:
# função para criar o database
def create_db(conexao, query):
    cursor = conexao.cursor(buffered=True)
    try:
        cursor.execute(query)
        print("Database created successfully")
    except Error as err:
        print(f"Error: '{err}'")

In [7]:
# função para conectar ao banco
# vou precisar do nome da conexao(servidor), nome da instancia(host/banco), usuário(user) e senha(password)

def conexao_db(servidor,usuario,senha,banco):
  conexao=None
  try:
    conexao = mysql.connector.connect(
        host = servidor,
        user = usuario,
        passwd = senha,
        database = banco
    )
    print('Conexao com o banco',banco,'realizado com sucesso!')
  except mysql.connector.Error as err:
    print('Erro ao conectar com o banco',banco,' erro:',err)
  return conexao


In [8]:
# função para poder executar os comandos do sql no python

def executar_sql(conexao, sql):
  cursor = conexao.cursor(buffered=True)
  try:
    cursor.execute(sql)
    conexao.commit() #para alterar no banco
    print('Query executada com sucesso!')
  except mysql.connector.Error as err:
    print('Erro ao executar a Query!',err)

In [29]:
# função para consultar as tabelas

def consulta_query(conexao,sql):
    cursor = conexao.cursor(buffered=True)
    result = None
    try:
        cursor.execute(sql)
        result = cursor.fetchall()
        return result
    except mysql.connector.Error as err:
        print('Erro ao executar a Query!',err)

# Criando o banco

In [12]:
# conectando ao servidor do banco
con = create_conexao('ip_do_seu_banco','root','senha')

MySQL Database connection successful


In [ ]:
# criando o banco rox
create_db_query="CREATE DATABASE dbrox"
create_db=(con,create_db_query)

In [30]:
# conectando ao banco de dados
con = conexao_db('ip_do_seu_banco','root','senha','dbrox')

Conexao com o banco dbrox realizado com sucesso!


# Criando e populando a tabela Person

Obs.: Para o processo de criação e inserção de dados na tabela, vou utilizar outro conector porque vou enviar o arquivo completo para o banco

In [16]:
# Escolher o arquivo dentro da bucket
bucket.blob('part-00000-39aaa492-9d87-419f-8b41-ac84b54e615d-c000.csv')

# Escolher uma variável que vai receber o caminho do arquivo que quero ler
path='gs://bucket-rox/Dados_Tratados/Person/part-00000-39aaa492-9d87-419f-8b41-ac84b54e615d-c000.csv'

In [17]:
# Lendo para o pandas
dfPerson = pd.read_csv(path)

In [ ]:
dfPerson

In [20]:
# coenctor para enviar todo o dataframe
engine = create_engine('mysql+pymysql://root:senha@ip_do_seu_banco:3306/dbrox')

In [21]:
dfPerson = dfPerson.to_sql('person',con=engine,if_exists='replace',index=False)

In [35]:
# incluindo a Primary Key na tabela
alter_person_table = ("ALTER TABLE person ADD CONSTRAINT PK_Person PRIMARY KEY (BusinessEntityID);")
executar_sql(con, alter_person_table)

Query executada com sucesso!


# Criando, populando e adicionando chaves a tabela Customer

In [36]:
# Escolher o arquivo dentro da bucket
bucket.blob('part-00000-acba03d6-caf7-45e4-98f9-e78dfb371080-c000.csv')

# Escolher uma variável que vai receber o caminho do arquivo que quero ler
path='gs://bucket-rox/Dados_Tratados/SalesCustomer/part-00000-acba03d6-caf7-45e4-98f9-e78dfb371080-c000.csv'

In [38]:
# Lendo para o pandas
dfCustomer = pd.read_csv(path)

In [39]:
dfCustomer

,CustomerID,PersonID,StoreID,TerritoryID,AccountNumber,rowguid,ModifiedDate
0,1,NaN,934.0,1,AW00000001,3F5AE95E-B87D-4AED-95B4-C3797AFCB74F,2014-09-12T11:15:07.263Z
1,2,NaN,1028.0,1,AW00000002,E552F657-A9AF-4A7D-A645-C429D6E02491,2014-09-12T11:15:07.263Z
2,3,NaN,642.0,4,AW00000003,130774B1-DB21-4EF3-98C8-C104BCD6ED6D,2014-09-12T11:15:07.263Z
3,4,NaN,932.0,4,AW00000004,FF862851-1DAA-4044-BE7C-3E85583C054D,2014-09-12T11:15:07.263Z
4,5,NaN,1026.0,4,AW00000005,83905BDC-6F5E-4F71-B162-C98DA069F38A,2014-09-12T11:15:07.263Z
...,...,...,...,...,...,...,...
19815,30114,1985.0,1986.0,7,AW00030114,97154F3D-28F1-4B15-AE03-9518B781ECE3,2014-09-12T11:15:07.263Z
19816,30115,1987.0,1988.0,6,AW00030115,E4CF8FD5-30A4-4B8E-8FD8-47032E255778,2014-09-12T11:15:07.263Z
19817,30116,1989.0,1990.0,4,AW00030116,EC409609-D25D-41B8-9D15-A1AA6E89FC77,2014-09-12T11:15:07.263Z
19818,30117,1991.0,1992.0,4,AW00030117,6F08E2FB-1CD3-4F6E-A2E6-385669598B19,2014-09-12T11:15:07.263Z


In [40]:
# conector para enviar todo o dataframe
engine = create_engine('mysql+pymysql://root:senha@ip_do_seu_banco:3306/dbrox')

In [41]:
dfCustomer = dfCustomer.to_sql('customer',con=engine,if_exists='replace',index=False)

In [43]:
# incluindo a Primary Key na tabela
alter_customer_table = ("ALTER TABLE customer ADD CONSTRAINT PK_Customer PRIMARY KEY (CustomerID);")
executar_sql(con, alter_customer_table)

Query executada com sucesso!


In [44]:
# modificando o tipo PersonID para Bigint
modify_customer_table = ("ALTER TABLE customer MODIFY PersonID BIGINT(11);")
executar_sql(con, modify_customer_table)

Query executada com sucesso!


In [46]:
# incluindo a foreing key tabela customer
alter_customer_table = ("ALTER TABLE customer ADD CONSTRAINT fk_customer FOREIGN KEY (PersonID) REFERENCES person(BusinessEntityID);")
executar_sql(con, alter_customer_table)

Query executada com sucesso!


# Criando, populando e adicionando chaves a tabela salesorderheader

In [41]:
# Escolher o arquivo dentro da bucket
bucket.blob('part-00000-69a7ed73-ab25-4571-86a6-a0d7b9a12600-c000.csv')

# Escolher uma variável que vai receber o caminho do arquivo que quero ler
path='gs://bucket-rox/Dados_Tratados/SalesOrderHeader/part-00000-69a7ed73-ab25-4571-86a6-a0d7b9a12600-c000.csv'

In [43]:
# Lendo para o pandas
dfSalesOrderHeader = pd.read_csv(path,parse_dates=['OrderDate'])

In [44]:
dfSalesOrderHeader

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,...,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43659,8,2011-05-31,2011-06-12T00:00:00.000Z,2011-06-07T00:00:00.000Z,5,0,SO43659,PO522145787,10-4020-000676,...,16281.0,105041Vi84182,NaN,"20565,6206","1971,5149","616,0984","23153,2339",NaN,79B65321-39CA-4115-9CBA-8FE0903E12E6,2011-06-07T00:00:00.000Z
1,43660,8,2011-05-31,2011-06-12T00:00:00.000Z,2011-06-07T00:00:00.000Z,5,0,SO43660,PO18850127500,10-4020-000117,...,5618.0,115213Vi29411,NaN,"1294,2529","124,2483","38,8276","1457,3288",NaN,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07T00:00:00.000Z
2,43661,8,2011-05-31,2011-06-12T00:00:00.000Z,2011-06-07T00:00:00.000Z,5,0,SO43661,PO18473189620,10-4020-000442,...,1346.0,85274Vi6854,4.0,"32726,4786","3153,7696","985,553","36865,8012",NaN,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07T00:00:00.000Z
3,43662,8,2011-05-31,2011-06-12T00:00:00.000Z,2011-06-07T00:00:00.000Z,5,0,SO43662,PO18444174044,10-4020-000227,...,10456.0,125295Vi53935,4.0,"28832,5289","2775,1646","867,2389","32474,9324",NaN,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07T00:00:00.000Z
4,43663,8,2011-05-31,2011-06-12T00:00:00.000Z,2011-06-07T00:00:00.000Z,5,0,SO43663,PO18009186470,10-4020-000510,...,4322.0,45303Vi22691,NaN,"419,4589","40,2681","12,5838","472,3108",NaN,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07T00:00:00.000Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31460,75119,8,2014-06-30,2014-07-12T00:00:00.000Z,2014-07-07T00:00:00.000Z,5,1,SO75119,NaN,10-4030-011981,...,6761.0,429826Vi35166,NaN,"42,28","3,3824","1,057","46,7194",NaN,9382F1C9-383A-435F-9449-0EECEA21B78D,2014-07-07T00:00:00.000Z
31461,75120,8,2014-06-30,2014-07-12T00:00:00.000Z,2014-07-07T00:00:00.000Z,5,1,SO75120,NaN,10-4030-018749,...,8925.0,929849Vi46003,NaN,"84,96","6,7968","2,124","93,8808",NaN,AE6A4FCF-FF73-4CD4-AF2C-5993D00D4AFE,2014-07-07T00:00:00.000Z
31462,75121,8,2014-06-30,2014-07-12T00:00:00.000Z,2014-07-07T00:00:00.000Z,5,1,SO75121,NaN,10-4030-015251,...,14220.0,529864Vi73738,NaN,"74,98","5,9984","1,8745","82,8529",NaN,D7395C0E-00CB-4BFA-A238-0D6A9F49884F,2014-07-07T00:00:00.000Z
31463,75122,8,2014-06-30,2014-07-12T00:00:00.000Z,2014-07-07T00:00:00.000Z,5,1,SO75122,NaN,10-4030-015868,...,18719.0,330022Vi97312,NaN,"30,97","2,4776","0,7743","34,2219",NaN,4221035A-4159-492F-AF40-4363A64FFC16,2014-07-07T00:00:00.000Z


In [45]:
# conector para enviar todo o dataframe
engine = create_engine('mysql+pymysql://root:senha@ip_do_seu_banco:3306/dbrox')

In [50]:
dfSalesOrderHeader = dfSalesOrderHeader.to_sql('salesorderheader',con=engine,if_exists='replace',index=False)

In [51]:
# incluindo a Primary Key na tabela
alter_salesorderheader_table = ("ALTER TABLE salesorderheader ADD CONSTRAINT fk_salesorderheade PRIMARY KEY (SalesOrderID);")
executar_sql(con, alter_salesorderheader_table)

Query executada com sucesso!


In [52]:
# incluindo a Primary Key na tabela
alter_salesorderheader_table = ("ALTER TABLE salesorderheader ADD CONSTRAINT fk_customerid FOREIGN KEY (CustomerID) REFERENCES customer(CustomerID);")
executar_sql(con, alter_salesorderheader_table)

Query executada com sucesso!


In [57]:
# alterando a coluna OrderDate
alter_salesorderheader_table = ("ALTER TABLE salesorderheader MODIFY OrderDate DATE ;")
executar_sql(con, alter_salesorderheader_table)

Query executada com sucesso!


# Criando, populando e adicionando chaves a tabela Production

In [25]:
# Escolher o arquivo dentro da bucket
bucket.blob('part-00000-5ace3503-c338-4857-857d-55c1bbd3d776-c000.cs')

# Escolher uma variável que vai receber o caminho do arquivo que quero ler
path='gs://bucket-rox/Dados_Tratados/Production/part-00000-5ace3503-c338-4857-857d-55c1bbd3d776-c000.csv'

In [26]:
# Lendo para o pandas
dfProduction = pd.read_csv(path)

In [27]:
dfProduction

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,...,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,0,0,NaN,1000,750,"0,00","0,00",...,NaN,NaN,NaN,NaN,NaN,2008-04-30T00:00:00.000Z,NaN,NaN,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08T10:01:36.827Z
1,2,Bearing Ball,BA-8327,0,0,NaN,1000,750,"0,00","0,00",...,NaN,NaN,NaN,NaN,NaN,2008-04-30T00:00:00.000Z,NaN,NaN,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08T10:01:36.827Z
2,3,BB Ball Bearing,BE-2349,1,0,NaN,800,600,"0,00","0,00",...,NaN,NaN,NaN,NaN,NaN,2008-04-30T00:00:00.000Z,NaN,NaN,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08T10:01:36.827Z
3,4,Headset Ball Bearings,BE-2908,0,0,NaN,800,600,"0,00","0,00",...,NaN,NaN,NaN,NaN,NaN,2008-04-30T00:00:00.000Z,NaN,NaN,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08T10:01:36.827Z
4,316,Blade,BL-2036,1,0,NaN,800,600,"0,00","0,00",...,NaN,NaN,NaN,NaN,NaN,2008-04-30T00:00:00.000Z,NaN,NaN,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08T10:01:36.827Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,995,ML Bottom Bracket,BB-8107,1,1,NaN,500,375,"44,9506","101,24",...,NaN,M,NaN,5.0,96.0,2013-05-30T00:00:00.000Z,NaN,NaN,71AB847F-D091-42D6-B735-7B0C2D82FC84,2014-02-08T10:01:36.827Z
500,996,HL Bottom Bracket,BB-9108,1,1,NaN,500,375,"53,9416","121,49",...,NaN,H,NaN,5.0,97.0,2013-05-30T00:00:00.000Z,NaN,NaN,230C47C5-08B2-4CE3-B706-69C0BDD62965,2014-02-08T10:01:36.827Z
501,997,"Road-750 Black, 44",BK-R19B-44,1,1,Black,100,75,"343,6496","539,99",...,R,L,U,2.0,31.0,2013-05-30T00:00:00.000Z,NaN,NaN,44CE4802-409F-43AB-9B27-CA53421805BE,2014-02-08T10:01:36.827Z
502,998,"Road-750 Black, 48",BK-R19B-48,1,1,Black,100,75,"343,6496","539,99",...,R,L,U,2.0,31.0,2013-05-30T00:00:00.000Z,NaN,NaN,3DE9A212-1D49-40B6-B10A-F564D981DBDE,2014-02-08T10:01:36.827Z


In [28]:
# conector para enviar todo o dataframe
engine = create_engine('mysql+pymysql://root:senha@ip_do_seu_banco:3306/dbrox')

In [29]:
dfProduction = dfProduction.to_sql('production',con=engine,if_exists='replace',index=False)

In [ ]:
# incluindo a Primary Key na tabela
alter_production_table = ("ALTER TABLE production ADD CONSTRAINT PK_product PRIMARY KEY (ProductID);")
executar_sql(con, alter_production_table)

# Criando, populando e adicionando chaves a tabela specialofferproduct

In [20]:
# Escolher o arquivo dentro da bucket
bucket.blob('part-00000-4abcde36-9d85-48f0-a916-5c95a43e76aa-c000.csv')

# Escolher uma variável que vai receber o caminho do arquivo que quero ler
path='gs://bucket-rox/Dados_Tratados/SpecialOfferProduct/part-00000-4abcde36-9d85-48f0-a916-5c95a43e76aa-c000.csv'

In [21]:
# Lendo para o pandas
dfSpecialOfferProduct = pd.read_csv(path)

In [22]:
dfSpecialOfferProduct

,SpecialOfferID,ProductID,rowguid,ModifiedDate
0,1,680,BB30B868-D86C-4557-8DB2-4B2D0A83A0FB,2011-04-01T00:00:00.000Z
1,1,706,B3C9A4B1-2AE6-4CBA-B552-1F206C9F4C1F,2011-04-01T00:00:00.000Z
2,1,707,27B711FE-0B77-4EA4-AD1A-7C239956BEF4,2011-04-01T00:00:00.000Z
3,1,708,46CBB78B-246E-4D69-9BD6-E521277C1078,2011-04-01T00:00:00.000Z
4,1,709,CF102AA0-055F-4D2B-8B98-04B161758EA8,2011-04-01T00:00:00.000Z
...,...,...,...,...
533,16,984,52DD0F65-0EA9-4848-9884-7594BA953274,2014-03-01T00:00:00.000Z
534,16,985,C8127A82-05C3-42FD-A5C6-215D58A236BD,2014-03-01T00:00:00.000Z
535,16,986,C60FB28D-2431-481C-985E-3FEA8119D705,2014-03-01T00:00:00.000Z
536,16,987,BEBD6F20-D6C2-4511-8C97-1F53B2099D1F,2014-03-01T00:00:00.000Z


In [23]:
# conector para enviar todo o dataframe
engine = create_engine('mysql+pymysql://root:senha@ip_do_seu_banco:3306/dbrox')

In [24]:
dfSpecialOfferProduct = dfSpecialOfferProduct.to_sql('specialofferproduct',con=engine,if_exists='replace',index=False)

In [10]:
# incluindo a Primary Key na tabela
alter_specialofferproduct_table = ("ALTER TABLE specialofferproduct ADD CONSTRAINT fk_production FOREIGN KEY (ProductID) REFERENCES production(ProductID);")
executar_sql(con, alter_specialofferproduct_table)

Query executada com sucesso!


# Criando, populando e adicionando chaves a tabela salesorderdetail
  

In [11]:
# Escolher o arquivo dentro da bucket
bucket.blob('part-00000-95723d44-f7da-450c-a651-a9d839068e92-c000.csv')

# Escolher uma variável que vai receber o caminho do arquivo que quero ler
path='gs://bucket-rox/Dados_Tratados/SalesOrderDetail/part-00000-95723d44-f7da-450c-a651-a9d839068e92-c000.csv'

In [12]:
# Lendo para o pandas
dfSalesOrderDetail = pd.read_csv(path)

In [13]:
dfSalesOrderDetail

,SalesOrderID,SalesOrderDetailID,CarrierTrackingNumber,OrderQty,ProductID,SpecialOfferID,UnitPrice,UnitPriceDiscount,LineTotal,rowguid,ModifiedDate
0,43659,1,4911-403C-98,1,776,1,"2024,994","0,00",2024.994,B207C96D-D9E6-402B-8470-2CC176C42283,2011-05-31T00:00:00.000Z
1,43659,2,4911-403C-98,3,777,1,"2024,994","0,00",6074.982,7ABB600D-1E77-41BE-9FE5-B9142CFC08FA,2011-05-31T00:00:00.000Z
2,43659,3,4911-403C-98,1,778,1,"2024,994","0,00",2024.994,475CF8C6-49F6-486E-B0AD-AFC6A50CDD2F,2011-05-31T00:00:00.000Z
3,43659,4,4911-403C-98,1,771,1,"2039,994","0,00",2039.994,04C4DE91-5815-45D6-8670-F462719FBCE3,2011-05-31T00:00:00.000Z
4,43659,5,4911-403C-98,1,772,1,"2039,994","0,00",2039.994,5A74C7D2-E641-438E-A7AC-37BF23280301,2011-05-31T00:00:00.000Z
...,...,...,...,...,...,...,...,...,...,...,...
121312,75122,121313,NaN,1,878,1,"21,98","0,00",21.980,8CAD6675-18CC-4F47-8287-97B41A8EE47D,2014-06-30T00:00:00.000Z
121313,75122,121314,NaN,1,712,1,"8,99","0,00",8.990,84F1C363-1C50-4442-BE16-541C59B6E12C,2014-06-30T00:00:00.000Z
121314,75123,121315,NaN,1,878,1,"21,98","0,00",21.980,C18B6476-429F-4BB1-828E-2BE5F82A0A51,2014-06-30T00:00:00.000Z
121315,75123,121316,NaN,1,879,1,"159,00","0,00",159.000,75A89C6A-C60A-47EA-8A52-B52A9C435B64,2014-06-30T00:00:00.000Z


In [27]:
# conector para enviar todo o dataframe
engine = create_engine('mysql+pymysql://root:senha@ip_do_seu_banco:3306/dbrox')

In [15]:
dfSalesOrderDetail = dfSalesOrderDetail.to_sql('salesorderdetail',con=engine,if_exists='replace',index=False)

In [16]:
# incluindo a Primary Key na tabela
alter_salesorderdetail_table = ("ALTER TABLE salesorderdetail ADD CONSTRAINT pk_salesorderdetail PRIMARY KEY (SalesOrderDetailID) ;")
executar_sql(con, alter_salesorderdetail_table)

Query executada com sucesso!


In [53]:
# incluindo a Primary Key na tabela
alter_salesorderdetail_table = ("ALTER TABLE salesorderdetail ADD CONSTRAINT fk_salesorder FOREIGN KEY (SalesOrderID) REFERENCES salesorderheader(SalesOrderID) ;")
executar_sql(con, alter_salesorderdetail_table)

Query executada com sucesso!


In [18]:
# incluindo a Primary Key na tabela
alter_salesorderdetail_table = ("ALTER TABLE salesorderdetail ADD CONSTRAINT fk_detilproduct FOREIGN KEY (ProductID) REFERENCES production(ProductID) ;")
executar_sql(con, alter_salesorderdetail_table)

Query executada com sucesso!


# Consultas

In [73]:
""" 1. Escreva uma query que retorna a quantidade de linhas na tabela Sales.SalesOrderDetail
pelo campo SalesOrderID, desde que tenham pelo menos três linhas de detalhes.
"""

q1 = """
SELECT SalesOrderID, COUNT(SalesOrderDetailID) \
FROM salesorderdetail \
GROUP by SalesOrderID \
HAVING COUNT(SalesOrderDetailID) > 3;
"""

results = consulta_query(con, q1)

for result in results:
  print(result)

A saída de streaming foi truncada nas últimas 5000 linhas.
(51684, 5)
(51690, 4)
(51691, 38)
(51693, 31)
(51694, 7)
(51696, 7)
(51698, 24)
(51699, 29)
(51700, 24)
(51703, 38)
(51704, 49)
(51705, 18)
(51706, 12)
(51708, 22)
(51710, 13)
(51711, 55)
(51712, 49)
(51714, 52)
(51715, 31)
(51716, 20)
(51717, 12)
(51718, 22)
(51719, 12)
(51720, 50)
(51721, 72)
(51722, 31)
(51723, 7)
(51724, 16)
(51726, 26)
(51728, 20)
(51729, 26)
(51730, 5)
(51731, 21)
(51733, 30)
(51734, 54)
(51735, 34)
(51736, 8)
(51737, 7)
(51739, 72)
(51740, 23)
(51741, 7)
(51742, 21)
(51743, 4)
(51745, 30)
(51746, 8)
(51747, 4)
(51748, 64)
(51749, 4)
(51750, 16)
(51751, 65)
(51752, 44)
(51753, 20)
(51755, 18)
(51757, 5)
(51758, 4)
(51760, 24)
(51761, 58)
(51762, 30)
(51763, 52)
(51764, 16)
(51766, 11)
(51769, 33)
(51770, 13)
(51771, 44)
(51772, 33)
(51773, 48)
(51774, 62)
(51775, 48)
(51776, 33)
(51777, 11)
(51778, 19)
(51780, 23)
(51781, 23)
(51783, 43)
(51784, 6)
(51785, 4)
(51786, 28)
(51789, 46)
(51790, 18)
(51791, 41

In [72]:
""" 2. Escreva uma query que ligue as tabelas Sales.SalesOrderDetail, Sales.SpecialOfferProduct e
Production.Product e retorne os 3 produtos (Name) mais vendidos (pela soma de OrderQty),
agrupados pelo número de dias para manufatura (DaysToManufacture).
Obs.: Entendo que deve ser produtos vendido em promoção. Consulta lenta.
"""

q2 = """
SELECT production.Name, production.DaysToManufacture, SUM(salesorderdetail.OrderQty) \
FROM salesorderdetail \
INNER JOIN specialofferproduct \
ON (salesorderdetail.SpecialOfferID=specialofferproduct.SpecialOfferID \
AND salesorderdetail.ProductID=specialofferproduct.ProductID) \
INNER JOIN production \
ON specialofferproduct.ProductID=production.ProductID \
GROUP BY production.Name, production.DaysToManufacture \
LIMIT 3;
"""

results = consulta_query(con, q2)

for result in results:
  print(result)

('Sport-100 Helmet, Red', 0, Decimal('6266'))
('Sport-100 Helmet, Black', 0, Decimal('6532'))
('Mountain Bike Socks, M', 0, Decimal('1107'))


In [70]:
""" 3. Escreva uma query ligando as tabelas Person.Person, Sales.Customer e Sales.SalesOrderHeader de
forma a obter uma lista de nomes de clientes e uma contagem de pedidos efetuados. """

q3 = """
SELECT person.Title, person.FirstName, person.LastName, count(salesorderheader.SalesOrderID) \
 FROM salesorderheader \
 INNER JOIN customer \
 ON salesorderheader.CustomerID=customer.CustomerID \
 INNER JOIN person \
 ON customer.PersonID=person.BusinessEntityID \
 GROUP BY person.Title, person.FirstName, person.LastName;
"""

results = consulta_query(con, q3)

for result in results:
  print(result)

A saída de streaming foi truncada nas últimas 5000 linhas.
(None, 'Kevin', 'Collins', 2)
(None, 'Diane', 'Suarez', 1)
(None, 'Billy', 'Gutierrez', 1)
(None, 'Gabriel', 'Collins', 1)
(None, 'Billy', 'Torres', 3)
(None, 'Billy', 'Dominguez', 1)
(None, 'Elijah', 'Collins', 1)
(None, 'Billy', 'Gill', 2)
(None, 'Jason', 'Collins', 1)
(None, 'Luis', 'Collins', 2)
(None, 'Aaron', 'Collins', 2)
(None, 'Billy', 'Blanco', 2)
(None, 'Billy', 'Serrano', 1)
(None, 'Caleb', 'Collins', 1)
(None, 'Connor', 'Collins', 1)
(None, 'Billy', 'Carlson', 1)
(None, 'Billy', 'Suarez', 2)
(None, 'Diane', 'Sanz', 1)
(None, 'Billy', 'Schmidt', 1)
(None, 'Luke', 'Collins', 1)
(None, 'Billy', 'Rubio', 1)
(None, 'Billy', 'Ortega', 1)
(None, 'Hector', 'Martin', 1)
(None, 'Hector', 'Gomez', 1)
(None, 'Jack', 'Collins', 2)
(None, 'Adam', 'Collins', 1)
(None, 'Martin', 'Rodriguez', 1)
(None, 'Eric', 'Collins', 1)
(None, 'Martin', 'Sanchez', 1)
(None, 'Jackson', 'Collins', 1)
(None, 'Martin', 'Perez', 2)
(None, 'Carlos', 

In [68]:
""" 4. Escreva uma query usando as tabelas Sales.SalesOrderHeader, Sales.SalesOrderDetail e Production.
Product, de forma a obter a soma total de produtos (OrderQty) por ProductID e OrderDate.
"""

q4 = """
SELECT production.ProductID, production.Name, salesorderheader.OrderDate, sum(salesorderdetail.OrderQty) \
 FROM salesorderheader \
 inner JOIN salesorderdetail \
 ON salesorderdetail.SalesOrderID = salesorderheader.SalesOrderID \
 INNER JOIN production \
 ON salesorderdetail.ProductID=production.ProductID \
 GROUP BY production.ProductID, production.Name, salesorderheader.OrderDate \
 ORDER BY production.Name, salesorderheader.OrderDate;
"""

results = consulta_query(con, q4)

for result in results:
  print(result)

A saída de streaming foi truncada nas últimas 5000 linhas.
(708, 'Sport-100 Helmet, Black', datetime.date(2013, 8, 22), Decimal('4'))
(708, 'Sport-100 Helmet, Black', datetime.date(2013, 8, 23), Decimal('9'))
(708, 'Sport-100 Helmet, Black', datetime.date(2013, 8, 24), Decimal('2'))
(708, 'Sport-100 Helmet, Black', datetime.date(2013, 8, 25), Decimal('5'))
(708, 'Sport-100 Helmet, Black', datetime.date(2013, 8, 26), Decimal('8'))
(708, 'Sport-100 Helmet, Black', datetime.date(2013, 8, 27), Decimal('3'))
(708, 'Sport-100 Helmet, Black', datetime.date(2013, 8, 28), Decimal('3'))
(708, 'Sport-100 Helmet, Black', datetime.date(2013, 8, 29), Decimal('12'))
(708, 'Sport-100 Helmet, Black', datetime.date(2013, 8, 30), Decimal('166'))
(708, 'Sport-100 Helmet, Black', datetime.date(2013, 8, 31), Decimal('6'))
(708, 'Sport-100 Helmet, Black', datetime.date(2013, 9, 1), Decimal('6'))
(708, 'Sport-100 Helmet, Black', datetime.date(2013, 9, 2), Decimal('3'))
(708, 'Sport-100 Helmet, Black', datetim

In [60]:
""" 5. Escreva uma query mostrando os campos SalesOrderID, OrderDate e TotalDue da tabela
Sales.SalesOrderHeader. Obtenha apenas as linhas onde a ordem tenha sido feita durante o mês de
setembro/2011 e o total devido esteja acima de 1.000. Ordene pelo total devido decrescente.
"""

q5 = """
SELECT SalesOrderID, OrderDate, TotalDue FROM salesorderheader where OrderDate between '2011-09-01' and '2011-09-30' and TotalDue > 1000 ORDER BY TotalDue DESC;
"""

results = consulta_query(con, q5)

for result in results:
  print(result)

(44324, datetime.date(2011, 9, 1), '3953,9884')
(44478, datetime.date(2011, 9, 29), '3953,9884')
(44326, datetime.date(2011, 9, 1), '3953,9884')
(44327, datetime.date(2011, 9, 2), '3953,9884')
(44328, datetime.date(2011, 9, 2), '3953,9884')
(44329, datetime.date(2011, 9, 2), '3953,9884')
(44330, datetime.date(2011, 9, 2), '3953,9884')
(44331, datetime.date(2011, 9, 3), '3953,9884')
(44332, datetime.date(2011, 9, 3), '3953,9884')
(44477, datetime.date(2011, 9, 29), '3953,9884')
(44334, datetime.date(2011, 9, 4), '3953,9884')
(44476, datetime.date(2011, 9, 29), '3953,9884')
(44473, datetime.date(2011, 9, 29), '3953,9884')
(44338, datetime.date(2011, 9, 4), '3953,9884')
(44339, datetime.date(2011, 9, 4), '3953,9884')
(44340, datetime.date(2011, 9, 4), '3953,9884')
(44472, datetime.date(2011, 9, 29), '3953,9884')
(44343, datetime.date(2011, 9, 5), '3953,9884')
(44344, datetime.date(2011, 9, 6), '3953,9884')
(44345, datetime.date(2011, 9, 6), '3953,9884')
(44347, datetime.date(2011, 9, 6), 